In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from vamos import (
    optimize, OptimizeConfig,
    NSGAIIConfig, NSGAIIIConfig, MOEADConfig,
    SPEA2Config, SMSEMOAConfig, IBEAConfig, SMPSOConfig,
    ZDT1, DTLZ2,
)
from vamos.algorithms import available_algorithms
from vamos.foundation.metrics import hypervolume

plt.style.use("ggplot")
print(f"Available algorithms: {available_algorithms()}")

## 1. Algorithm Configurations

Each algorithm has its own config builder with algorithm-specific parameters.

In [ ]:
# Common parameters
POP_SIZE = 100
MAX_EVALS = 10000

# Algorithm-specific configurations
configs = {
    "nsgaii": NSGAIIConfig()
        .pop_size(56)
        .offspring_size(14)
        .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
        .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
        .selection("tournament", pressure=9)
        .repair("round")
        .external_archive(size=56, archive_type="hypervolume")
        .engine("numpy")
        .fixed(),
    
    "moead": MOEADConfig()
        .pop_size(POP_SIZE)
        .n_neighbors(20)
        .decomposition("tchebycheff")
        .crossover("sbx", prob=0.9, eta=20.0)
        .mutation("pm", prob="1/n", eta=20.0)
        .engine("numpy")
        .fixed(),
    
    "spea2": SPEA2Config()
        .pop_size(POP_SIZE)
        .archive_size(POP_SIZE)
        .crossover("sbx", prob=0.9, eta=20.0)
        .mutation("pm", prob="1/n", eta=20.0)
        .engine("numpy")
        .fixed(),
    
    "smsemoa": SMSEMOAConfig()
        .pop_size(POP_SIZE)
        .crossover("sbx", prob=0.9, eta=20.0)
        .mutation("pm", prob="1/n", eta=20.0)
        .engine("numpy")
        .fixed(),
    
    "ibea": IBEAConfig()
        .pop_size(POP_SIZE)
        .crossover("sbx", prob=0.9, eta=20.0)
        .mutation("pm", prob="1/n", eta=20.0)
        .engine("numpy")
        .fixed(),
    
    "smpso": SMPSOConfig()
        .pop_size(POP_SIZE)
        .archive_size(POP_SIZE)
        .engine("numpy")
        .fixed(),
}

print(f"Configured {len(configs)} algorithms for comparison")


## 2. Bi-Objective Comparison (ZDT1)

In [ ]:
# Create ZDT1 problem
zdt1 = ZDT1(n_var=30)
ref_point = np.array([1.1, 1.1])  # For hypervolume calculation

# Run all algorithms
results = {}
runtimes = {}

for algo_name, config in configs.items():
    print(f"Running {algo_name.upper()}...", end=" ")
    start = time.time()
    
    result = optimize(OptimizeConfig(
        problem=zdt1,
        algorithm=algo_name,
        algorithm_config=config,
        termination=("n_eval", MAX_EVALS),
        seed=42,
    ))
    
    elapsed = time.time() - start
    results[algo_name] = result
    runtimes[algo_name] = elapsed

    F_front = result.front()
    F_pop = result.F
    hv_front = hypervolume(F_front, ref_point) if F_front is not None and F_front.size else float("nan")
    hv_pop = hypervolume(F_pop, ref_point) if F_pop is not None and F_pop.size else float("nan")
    n_front = len(F_front) if F_front is not None else 0
    n_pop = len(F_pop) if F_pop is not None else 0
    print(f"{n_front}/{n_pop} front/pop, HV_front={hv_front:.4f}, HV_pop={hv_pop:.4f}, time={elapsed:.2f}s")


In [ ]:
from vamos import pareto_filter

# Visualize union Pareto front
fig, ax = plt.subplots(figsize=(10, 7))

fronts = []
for result in results.values():
    F = result.front()
    if F is not None and F.size:
        fronts.append(F)

if fronts:
    F_all = np.vstack(fronts)
    F_plot = pareto_filter(F_all)
    if F_plot is not None and F_plot.size:
        ax.scatter(F_plot[:, 0], F_plot[:, 1], s=30, alpha=0.7, c='tab:blue', label='Union PF')

# True Pareto front
f1_true = np.linspace(0, 1, 100)
f2_true = 1 - np.sqrt(f1_true)
ax.plot(f1_true, f2_true, 'k--', linewidth=2, alpha=0.5, label='True PF')

ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.set_title(f"ZDT1: Algorithm Comparison ({MAX_EVALS} evaluations)")
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.05, 1.1)
ax.set_ylim(-0.05, 1.1)
plt.tight_layout()
plt.show()


In [ ]:
# Compare hypervolume values (front vs population)
hv_front = {}
hv_pop = {}
front_sizes = {}
pop_sizes = {}

for name, res in results.items():
    F_front = res.front()
    F_pop = res.F
    hv_front[name] = hypervolume(F_front, ref_point) if F_front is not None and F_front.size else float("nan")
    hv_pop[name] = hypervolume(F_pop, ref_point) if F_pop is not None and F_pop.size else float("nan")
    front_sizes[name] = len(F_front) if F_front is not None else 0
    pop_sizes[name] = len(F_pop) if F_pop is not None else 0

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Hypervolume bar chart (front only)
names = list(hv_front.keys())
hvs = [hv_front[name] for name in names]
colors_bar = plt.cm.viridis(np.linspace(0.2, 0.8, len(names)))

axes[0].bar(range(len(names)), hvs, color=colors_bar)
axes[0].set_xticks(range(len(names)))
axes[0].set_xticklabels([n.upper() for n in names], rotation=45, ha='right')
axes[0].set_ylabel("Hypervolume")
axes[0].set_title("Hypervolume (front)")
axes[0].set_ylim(min(hvs) * 0.95, max(hvs) * 1.02)

# Runtime bar chart
times = [runtimes[n] for n in names]
axes[1].bar(range(len(names)), times, color=colors_bar)
axes[1].set_xticks(range(len(names)))
axes[1].set_xticklabels([n.upper() for n in names], rotation=45, ha='right')
axes[1].set_ylabel("Runtime (seconds)")
axes[1].set_title("Runtime (lower = faster)")

plt.tight_layout()
plt.show()

# Summary table
print("")
print("Summary:")
print("-" * 80)
print(f"{'Algorithm':<12} {'HV(front)':>10} {'HV(pop)':>10} {'Runtime':>10} {'Front/Pop':>10}")
print("-" * 80)
for name in names:
    ratio = f"{front_sizes[name]}/{pop_sizes[name]}"
    print(
        f"{name.upper():<12} {hv_front[name]:>10.4f} {hv_pop[name]:>10.4f} {runtimes[name]:>9.2f}s {ratio:>10}"
    )


## 3. Algorithm Characteristics

### When to Use Each Algorithm

In [ ]:
characteristics = {
    "NSGA-II": {
        "strengths": ["Fast", "Well-studied", "Good spread"],
        "weaknesses": ["Struggles with >3 objectives"],
        "best_for": "2-3 objective problems, general use",
    },
    "NSGA-III": {
        "strengths": ["Many-objective capable", "Reference points"],
        "weaknesses": ["Needs reference point setup"],
        "best_for": "4+ objectives, structured Pareto fronts",
    },
    "MOEA/D": {
        "strengths": ["Explicit diversity", "Decomposition-based"],
        "weaknesses": ["Weight vector sensitive"],
        "best_for": "Convex fronts, uniform coverage",
    },
    "SPEA2": {
        "strengths": ["Fine-grained fitness", "Archive-based"],
        "weaknesses": ["Slower than NSGA-II"],
        "best_for": "When diversity is critical",
    },
    "SMS-EMOA": {
        "strengths": ["Direct HV optimization", "Theoretical convergence"],
        "weaknesses": ["Slow HV calculation"],
        "best_for": "High-quality fronts, small populations",
    },
    "IBEA": {
        "strengths": ["Indicator-driven", "Flexible indicators"],
        "weaknesses": ["Indicator choice matters"],
        "best_for": "When specific indicator quality needed",
    },
    "SMPSO": {
        "strengths": ["No crossover tuning", "Good for continuous"],
        "weaknesses": ["Only for continuous variables"],
        "best_for": "Continuous problems, different search dynamics",
    },
}

for algo, info in characteristics.items():
    print(f"\n{algo}:")
    print(f"  Strengths: {', '.join(info['strengths'])}")
    print(f"  Best for: {info['best_for']}")

## 4. NSGA-III for Many-Objective (DTLZ2)

In [ ]:
# Create 3-objective DTLZ2
dtlz2 = DTLZ2(n_var=12, n_obj=3)

print(f"DTLZ2: {dtlz2.n_var} variables, {dtlz2.n_obj} objectives")
print("\nNSGA-III is recommended for 3+ objectives due to reference points.")

In [ ]:
# Compare NSGA-II vs NSGA-III on 3-objective problem
nsgaii_3obj = (
    NSGAIIConfig()
    .pop_size(56)
    .offspring_size(14)
    .crossover("blx_alpha", prob=0.88, alpha=0.94, repair="clip")
    .mutation("non_uniform", prob="0.45/n", perturbation=0.3)
    .selection("tournament", pressure=9)
    .repair("round")
    .external_archive(size=56, archive_type="hypervolume")
    .engine("numpy")
    .fixed()
)
nsgaiii_3obj = NSGAIIIConfig().pop_size(100).n_partitions(12).engine("numpy").fixed()

print("Running NSGA-II on DTLZ2 (3 objectives)...")
result_nsgaii_3obj = optimize(OptimizeConfig(
    problem=dtlz2,
    algorithm="nsgaii",
    algorithm_config=nsgaii_3obj,
    termination=("n_eval", 15000),
    seed=42,
))

print("Running NSGA-III on DTLZ2 (3 objectives)...")
result_nsgaiii_3obj = optimize(OptimizeConfig(
    problem=dtlz2,
    algorithm="nsgaiii",
    algorithm_config=nsgaiii_3obj,
    termination=("n_eval", 15000),
    seed=42,
))

print(f"NSGA-II: {len(result_nsgaii_3obj.F)} solutions")
print(f"NSGA-III: {len(result_nsgaiii_3obj.F)} solutions")


In [ ]:
# 3D visualization
from mpl_toolkits.mplot3d import Axes3D
from vamos import pareto_filter

fig = plt.figure(figsize=(12, 5))

# Union front (NSGA-II + NSGA-III)
ax1 = fig.add_subplot(121, projection='3d')
fronts = []
for res in (result_nsgaii_3obj, result_nsgaiii_3obj):
    F = res.front()
    if F is not None and F.size:
        fronts.append(F)
if fronts:
    F_all = np.vstack(fronts)
    F_plot = pareto_filter(F_all)
    if F_plot is not None and F_plot.size:
        ax1.scatter(F_plot[:, 0], F_plot[:, 1], F_plot[:, 2], s=20, alpha=0.7, c='steelblue')
ax1.set_xlabel('f1')
ax1.set_ylabel('f2')
ax1.set_zlabel('f3')
ax1.set_title('Union PF')

# True Pareto front (unit sphere surface)
ax2 = fig.add_subplot(122, projection='3d')
u = np.linspace(0, np.pi/2, 30)
v = np.linspace(0, np.pi/2, 30)
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones(np.size(u)), np.cos(v))
ax2.plot_surface(x, y, z, alpha=0.3, color='gray')
ax2.set_xlabel('f1')
ax2.set_ylabel('f2')
ax2.set_zlabel('f3')
ax2.set_title('True PF (sphere)')

plt.tight_layout()
plt.show()


## 5. Statistical Comparison (Multiple Seeds)

In [ ]:
# Run multiple seeds for statistical comparison
N_SEEDS = 5
algorithms_to_compare = ["nsgaii", "moead", "spea2", "smsemoa"]

hv_per_seed = {algo: [] for algo in algorithms_to_compare}

print(f"Running {N_SEEDS} seeds for statistical comparison...")
print("-" * 50)

for seed in range(N_SEEDS):
    print(f"Seed {seed + 1}/{N_SEEDS}: ", end="")
    for algo in algorithms_to_compare:
        result = optimize(OptimizeConfig(
            problem=zdt1,
            algorithm=algo,
            algorithm_config=configs[algo],
            termination=("n_eval", 5000),  # Reduced for speed
            seed=seed,
        ))
        F_front = result.front()
        F_pop = result.F
        hv_front = hypervolume(F_front, ref_point) if F_front is not None and F_front.size else float("nan")
        hv_pop = hypervolume(F_pop, ref_point) if F_pop is not None and F_pop.size else float("nan")
        hv_per_seed[algo].append(hv_front)
        print(f"{algo}={hv_front:.4f} (pop {hv_pop:.4f}) ", end="")
    print()


In [ ]:
# Box plot comparison
fig, ax = plt.subplots(figsize=(8, 5))

data = [hv_per_seed[algo] for algo in algorithms_to_compare]
bp = ax.boxplot(data, labels=[a.upper() for a in algorithms_to_compare], patch_artist=True)

colors_box = ['steelblue', 'darkorange', 'forestgreen', 'purple']
for patch, color in zip(bp['boxes'], colors_box):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel("Hypervolume")
ax.set_title(f"ZDT1: Hypervolume Distribution ({N_SEEDS} seeds)")
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# Statistics summary
print("\nStatistics:")
print("-" * 40)
print(f"{'Algorithm':<12} {'Mean':>10} {'Std':>10}")
print("-" * 40)
for algo in algorithms_to_compare:
    hvs = hv_per_seed[algo]
    print(f"{algo.upper():<12} {np.mean(hvs):>10.4f} {np.std(hvs):>10.4f}")

## 6. Algorithm Selection Guide

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    ALGORITHM SELECTION GUIDE                         ║
╠══════════════════════════════════════════════════════════════════════╣
║ PROBLEM TYPE                  │ RECOMMENDED ALGORITHM                ║
╠═══════════════════════════════╪══════════════════════════════════════╣
║ 2-3 objectives, general       │ NSGA-II (fast, reliable)             ║
║ 4+ objectives (many-obj)      │ NSGA-III (reference points)          ║
║ Convex Pareto front           │ MOEA/D (decomposition)               ║
║ Need maximum diversity        │ SPEA2 (fine-grained fitness)         ║
║ Quality > speed               │ SMS-EMOA (hypervolume-driven)        ║
║ Continuous, no crossover      │ SMPSO (particle swarm)               ║
║ Indicator optimization        │ IBEA (flexible indicators)           ║
╠═══════════════════════════════╧══════════════════════════════════════╣
║ DEFAULT CHOICE: Start with NSGA-II, then try others if needed.       ║
╚══════════════════════════════════════════════════════════════════════╝
""")

## Summary

**Key Takeaways:**

1. **NSGA-II** is the default choice - fast, well-tested, works for most 2-3 objective problems
2. **NSGA-III** excels at many-objective (4+) problems with structured reference points
3. **MOEA/D** provides uniform coverage via decomposition (good for convex fronts)
4. **SPEA2** and **SMS-EMOA** prioritize quality over speed
5. **SMPSO** offers different search dynamics without crossover/mutation tuning
6. Always run multiple seeds for reliable comparisons

**Config Pattern:**
```python
config = AlgorithmConfig().pop_size(100).crossover(...).mutation(...).fixed()
result = optimize(OptimizeConfig(problem=p, algorithm="name", algorithm_config=config, ...))
```